In [4]:
import nltk

from nltk import CFG

In [5]:
groucho_grammar = CFG.fromstring("""
                                S -> NP VP
                                PP -> P NP
                                NP -> Det N | Det N PP | 'I'
                                VP -> V NP | VP PP
                                Det -> 'an' | 'my'
                                N -> 'elephant' | 'pajamas'
                                V -> 'shot'
                                P -> 'in'
                                """)

In [6]:
sent = 'I shot an elephant in my pajamas'
sent = nltk.word_tokenize(sent)
parser = nltk.ChartParser(groucho_grammar)
#trees = parser.nbest_parse(sent)
# AttributeError:'ChartParser' object has no attribute 'nbest_parse'
trees = parser.parse(sent)
for tree in trees:
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


In [10]:
grammar1 = CFG.fromstring("""
                          S -> NP VP
                          VP -> V NP | V NP PP
                          PP -> P NP
                          V -> "saw" | "ate" | "walked"
                          NP -> "John" | "Mary" | "Bob" | Det N | Det N PP 
                          Det -> "a" | "an" | "the" | "my"
                          N -> "man" | "dog" | "cat" | "telescope" | "park"
                          P -> "in" | "on" | "by" | "with"
                          """)

In [12]:
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


### 句法结构中的递归

In [15]:
grammar2 = CFG.fromstring("""
S ->NPVP
NP -> Det Nom | PropN
Nom -> Adj Nom | N
VP -> V Adj | V NP | V S | V NP PP
PP -> P NP
PropN -> 'Buster' | 'Chatterer' | 'Joe'
Det -> 'the' | 'a'
N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
Adj -> 'angry' | 'frightened' | 'little' | 'tall'
V -> 'chased' | 'saw' | 'said' | 'thought' | 'was' | 'put' 
P -> 'on'
""")

In [17]:
# NLTK提供的递归下降分析器
rd_parser = nltk.RecursiveDescentParser(grammar1) 
sent = 'Mary saw a dog'.split()
for t in rd_parser.parse(sent):
    print(t)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


In [21]:
sr_parse = nltk.ShiftReduceParser(grammar1)
sent = 'Mary saw a dog'.split()
for t in sr_parse.parse(sent):
    print(t)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


### 符合语句规则的子串表

In [22]:
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)] 
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i+1] = productions[0].lhs() 
    return wfst

In [24]:
def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions()) 
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span): 
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end] 
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end] = index[(nt1,nt2)] 
                    if trace:
                        print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
    return wfst

In [50]:
def display(wfst, tokens):
    print('\nWFST ' + ' '.join([("%-4d" % i) for i in range(1, len(wfst))])) 
    for i in range(len(wfst)-1):
        print("%d " % i, end='')
        for j in range(1, len(wfst)):
            print("%-5s" % (wfst[i][j] or '.'), end='')
        print()

In [51]:
tokens = "I shot an elephant in my pajamas".split()
wfst0 = init_wfst(tokens, groucho_grammar)
display(wfst0, tokens)


WFST 1    2    3    4    5    6    7   
0 NP   .    .    .    .    .    .    
1 .    V    .    .    .    .    .    
2 .    .    Det  .    .    .    .    
3 .    .    .    N    .    .    .    
4 .    .    .    .    P    .    .    
5 .    .    .    .    .    Det  .    
6 .    .    .    .    .    .    N    


In [52]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)

[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]


### 依存关系和依存文法

In [54]:
from nltk.grammar import DependencyGrammar
from nltk.parse import (
    DependencyGraph,
    ProjectiveDependencyParser,
    NonprojectiveDependencyParser,
    )
 
sent = 'I shot an elephant in my pajamas'.split()
dep_grammar = DependencyGrammar.fromstring(
    """
    'shot' -> 'I' | 'elephant' | 'in'
    'elephant' -> 'an' | 'in'
    'in' -> 'pajamas'
    'pajamas' -> 'my'
    """
    )
trees = ProjectiveDependencyParser(dep_grammar).parse(sent)
for tr in trees:
    print(tr)  
trees = ProjectiveDependencyParser(dep_grammar).parse(sent)
for tr in trees:
    tr.draw()

(shot I (elephant an (in (pajamas my))))
(shot I (elephant an) (in (pajamas my)))


In [55]:
from nltk.corpus import treebank
t = treebank.parsed_sents('wsj_0001.mrg')[0]
print(t)

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))


In [61]:
entries = nltk.corpus.ppattach.attachments('training') 
table = nltk.defaultdict(lambda: nltk.defaultdict(set)) 
for entry in entries:
    key = entry.noun1 + '-' + entry.prep + '-' + entry.noun2
    table[key][entry.attachment].add(entry.verb)

for key in sorted(table):
    if len(table[key]) > 1:
        print(key, 'N:', sorted(table[key]['N']), 'V:', sorted(table[key]['V']))

%-below-level N: ['left'] V: ['be']
%-from-year N: ['was'] V: ['declined', 'dropped', 'fell', 'grew', 'increased', 'plunged', 'rose', 'was']
%-in-August N: ['was'] V: ['climbed', 'fell', 'leaping', 'rising', 'rose']
%-in-September N: ['increased'] V: ['climbed', 'declined', 'dropped', 'edged', 'fell', 'grew', 'plunged', 'rose', 'slipped']
%-in-week N: ['declined'] V: ['was']
%-to-% N: ['add', 'added', 'backed', 'be', 'cut', 'go', 'grow', 'increased', 'increasing', 'is', 'offer', 'plummet', 'reduce', 'rejected', 'rise', 'risen', 'shaved', 'wants', 'yield', 'zapping'] V: ['fell', 'rise', 'slipped']
%-to-million N: ['declining'] V: ['advanced', 'climbed', 'cutting', 'declined', 'declining', 'dived', 'dropped', 'edged', 'fell', 'gained', 'grew', 'increased', 'jump', 'jumped', 'plunged', 'rising', 'rose', 'slid', 'slipped', 'soared', 'tumbled']
1-to-21 N: ['dropped'] V: ['dropped']
1-to-33 N: ['gained'] V: ['dropped', 'fell', 'jumped']
1-to-4 N: ['added'] V: ['gained']
1-to-47 N: ['jumped']

In [65]:
# 加权文法
def give(t):
    return t.node == 'VP' and len(t) > 2 and t[1].node == 'NP' and (t[2].node == 'PP-DTV' or t[2].node == 'NP') and ('give' in t[0].leaves() or 'gave' in t[0].leaves())
def sent(t):
    return ' '.join(token for token in t.leaves() if token[0] not in '*-0')
def print_node(t, width):
    output = "%s %s: %s / %s: %s" %(sent(t[0]), t[1].node, sent(t[1]), t[2].node, sent(t[2])) 
    if len(output) > width:
        output = output[:width] + "..." 
    print(output)

In [67]:
for tree in nltk.corpus.treebank.parsed_sents():
    for t in tree.subtrees(give):
        print_node(t, 72)

In [63]:
# 概率上下文无关文法
grammar = nltk.PCFG.fromstring("""
S -> NP VP   [1.0]
VP -> TV NP  [0.4]
VP ->IV      [0.3]
VP -> DatV NP NP [0.3]
TV -> 'saw'    [1.0]
IV -> 'ate'    [1.0]
DatV -> 'gave'    [1.0]
NP -> 'telescopes' [0.8]
NP -> 'Jack'  [0.2]
""")

In [64]:
print(grammar)

Grammar with 9 productions (start state = S)
    S -> NP VP [1.0]
    VP -> TV NP [0.4]
    VP -> IV [0.3]
    VP -> DatV NP NP [0.3]
    TV -> 'saw' [1.0]
    IV -> 'ate' [1.0]
    DatV -> 'gave' [1.0]
    NP -> 'telescopes' [0.8]
    NP -> 'Jack' [0.2]
